In [1]:
import random
import math
import numpy as np
import pandas as pd
type(np.int32(np.atleast_1d(np.int32(0))))
from operator import attrgetter
#import matplotlib.pyplot as plt
#from IPython.display import clear_output
print('ok')

ok


In [2]:
writing=open("GA_for_HAWK_Smax_2峰_ver1.csv","w",encoding="utf_8")

1

In [3]:
wavedate=pd.read_csv("HAWK_ver1.csv", encoding="shift_jis",engine="python")#
wavedata=wavedate.values[:,16:25*36+16]
estimated_parameters=pd.read_csv("esti_HAWK_Smax_ver1.csv", encoding="shift_jis",engine="python")#
esti_para=estimated_parameters.values

freq=np.array([0.0445953,0.0486315,0.053033,0.0578329,0.0630672,0.0687753,0.075,0.0817881,
               0.0891906,0.097263,0.106066,0.1156658,0.1261345,0.1375506,0.15,0.1635762,0.1783811,
               0.194526,0.2121321,0.2313317,0.252269,0.2751013,0.3000001,0.3271524,0.3567623])*2*math.pi
range_max=1.5
range_min=0.5
n_gene   = 96   # The number of genes.
n_ind    = 50  #the number of individuals in a population.
CXPB     = 0.85   # The probability of crossover.
MUTPB    = 1.0   # The probability of individdual mutation.
NGEN     = 2000   # The number of generation loop.
n_parents=2     #非復元抽出の個体数
MUTINDPB = 0.0050  # The probability of gene mutation.
CXPB_MAX =0.900   #交叉率の最大値
CXPB_MIN =0.200   #交叉率の最小値
M_MAX    =0.1#10.0/n_gene  #突然変異率の最大値
M_MIN    =0.01#1.0/n_gene/2  #突然変異率の最小値
f        =15
print(np.shape((wavedata)))
print(np.shape((esti_para)))

(3000, 900)
(3000, 13)


In [4]:
class Individual(np.ndarray):
    """Container of a individual."""
    fitness = None
    
    def __new__(cls, a):
        return np.asarray(a).view(cls)

def create_ind(n_gene,parameters):
    Hv_1,cos_1,sin_1,lambda_1,omega_1,S_max1=parameters[0],parameters[1],parameters[2],parameters[3],parameters[4],parameters[5]
    Hv_2,cos_2,sin_2,lambda_2,omega_2,S_max2=parameters[6],parameters[7],parameters[8],parameters[9],parameters[10],parameters[11]
    
    """Create a individual."""
    ind=np.zeros(n_gene)
    
    Hv_1 = Hv_1*np.random.uniform(0.7,1.3)
    Hv_1 = Hv_1 if Hv_1 <= 10.23 else 10.23
    Hv_1 = Hv_1 if Hv_1 >=0.01 else 0.01
    Hv_1='{:09b}'.format(int((Hv_1-0.01)/0.020))
    
    lambda_1 =lambda_1*np.random.uniform(0.4,1.6)
    lambda_1 =lambda_1 if lambda_1 <= 7.675 else 7.675#0.01+0.015*np.sum([ind[i+a]*2**i for i in range(8,-1,-1)])#0.01<=H<=7.675
    lambda_1 =lambda_1 if lambda_1 >= 0.01 else 0.01
    lambda_1='{:09b}'.format(int((lambda_1-0.01)/0.015))
    
    theta1  =np.arctan2(sin_1,cos_1)*180/np.pi+np.random.uniform(-10,10)
    cos_1   ='{:08b}'.format(int((np.cos(theta1*np.pi/180)+1)/2*255))#2/255*np.sum([ind[i+a]*2**i for i in range(7,-1,-1)])-1
    sin_1   ='{:08b}'.format(int((np.sin(theta1*np.pi/180)+1)/2*255))#2/255*np.sum([ind[i+a]*2**i for i in range(7,-1,-1)])-1
    
    omega_1 = omega_1*np.random.uniform(0.9,1.1)
    omega_1 = omega_1 if omega_1 <= 2.255 else 2.255
    omega_1 = omega_1 if omega_1 >= 0.35 else 0.35
    omega_1 ='{:07b}'.format(int((omega_1-0.35)/0.015))#0.35+0.015*np.sum([ind[i+a]*2**i for i in range(6,-1,-1)])#0.35<=omega<=2.255
    
    S_max1  = S_max1*np.random.uniform(0.6,1.4)
    S_max1  = S_max1 if S_max1 <= 84 else 84
    S_max1  = S_max1 if S_max1 >= 1 else 1
    S_max1  ='{:07b}'.format(int((S_max1-1)/83*127))
    
    
    """2峰目"""
    Hv_2 = Hv_2*np.random.uniform(0.5,1.5)
    Hv_2 = Hv_2 if Hv_2 <= 10.22 else 10.22#0<=H<=10.22
    Hv_2='{:09b}'.format(int(Hv_2/0.020))
    
    lambda_2 =lambda_2*np.random.uniform(0.4,1.6)
    lambda_2 =lambda_2 if lambda_2 <= 7.675 else 7.675
    lambda_2 =lambda_2 if lambda_2 >= 0.01 else 0.01
    lambda_2='{:09b}'.format(int((lambda_2-0.01)/0.015))
    
    theta2  =np.arctan2(sin_2,cos_2)*180/np.pi+np.random.uniform(-40,40)
    cos_2   ='{:08b}'.format(int((np.cos(theta2*np.pi/180)+1)/2*255))
    sin_2   ='{:08b}'.format(int((np.sin(theta2*np.pi/180)+1)/2*255))
    
    omega_2 = omega_2*np.random.uniform(0.6,1.4)
    omega_2 = omega_2 if omega_2 <= 2.255 else 2.255
    omega_2 = omega_2 if omega_2 >= 0.35 else 0.35
    omega_2  ='{:07b}'.format(int((omega_2-0.35)/0.015))
    
    S_max2  = S_max2*np.random.uniform(0.5,1.5)
    S_max2  = S_max2 if S_max2 <= 84 else 84
    S_max2  = S_max2 if S_max2 >= 1 else 1
    S_max2  ='{:07b}'.format(int((S_max2-1)/83*127))
    
    for i in range(n_gene):
        ind[i]=int((Hv_1+lambda_1+cos_1+sin_1+omega_1+S_max1+Hv_2+lambda_2+cos_2+sin_2+omega_2+S_max2)[i])
    
    return Individual(ind)#Individual([np.random.randint(0,2) for i in range(n_gene)])

def create_pop(n_ind, n_gene):
    """Create a population."""
    pop = []
    for i in range(n_ind):
        ind = create_ind(n_gene,parameters)
        pop.append(ind)
    return pop

def set_fitness(eval_func, pop):
    """Set fitnesses of each individual in a population."""
    for k, fit in zip(range(len(pop)), map(eval_func, pop)):
        pop[k].fitness = fit


def spectrum(Hv_1,theta_1,lambda_1,omega_1,s_1,theta_func,omega):
    """2π掛けてること忘れずに"""
    spectrum_11=0.250*((4.0*lambda_1+1.0)/4.0*omega_1**4.0)**lambda_1
    spectrum_12=math.gamma(lambda_1)
    spectrum_16=Hv_1**2.0
    spectrum_13=omega**(4.0*lambda_1+1.0)
    spectrum_14=2.0**(2.0*s_1-1.0)*(math.gamma(s_1+1.0))**2.0/math.pi/math.gamma(2.0*s_1+1.0)
    spectrum_15=abs((math.cos((theta_func-theta_1)/2.0*math.pi/180.0)))**(2.0*s_1)*math.exp(-(4.0*lambda_1+1.0)/4.0*(omega_1/omega)**4.0)
    return 2*math.pi*spectrum_11/spectrum_13*spectrum_14*spectrum_15*spectrum_16/spectrum_12


def evalOneMax(ind):
    """Objective function."""
    H1,cos1,sin1,lambda1,omega1,H2,cos2,sin2,lambda2,omega2,S_max1,S_max2 = get_parameter_from_ind(ind)
    theta_1=np.arctan2(sin1,cos1)*180/np.pi
    theta_2=np.arctan2(sin2,cos2)*180/np.pi
    s_1=[(S_max1*i**5/omega1**5) if i<=omega1 else (S_max1*i**(-2.5)/omega1**(-2.5)) for i in omega]
    s_2=[(S_max2*i**5/omega2**5) if i<=omega2 else (S_max2*i**(-2.5)/omega2**(-2.5)) for i in omega]
    return np.sum([[(seastate[36*i+j]-spectrum(H1,theta_1,lambda1,omega1,s_1[i],j*10,omega[i])-spectrum(H2,theta_2,lambda2,omega2,s_2[i],j*10,omega[i]))**2 for j in range(36)] for i in range(25)])#errorfunc

def adjustrating(C_p,M_p,CXPB_MAX,CXPB_MIN,M_MAX,M_MIN,f,f_max,fitness_mean):
    cross_para = 1.0/f-1/abs(f_max-fitness_mean)
    if (C_p*(1+cross_para))<=CXPB_MIN:
        C_p = CXPB_MIN
    else:
        C_p = C_p*(1+cross_para) if (C_p*(1+cross_para)) <= CXPB_MAX else CXPB_MAX    
                                  
    M_para = 1/abs(f_max-fitness_mean) - 1/f
    if (M_p*(1+M_para))<=M_MIN:
        M_p = M_MIN
    else:
        M_p = M_p*(1+M_para) if  (M_p*(1+M_para)) <= M_MAX else M_MAX
         
    return  C_p,M_p

def randomselection(n_ind,pop,n_parents):
    chosen=[]    
    for i in range(n_parents):
        idx=random.randrange(0, n_ind-i)
        chosen.append(pop[idx])
        pop.pop(idx)
    return chosen,pop

def tournament(pop, n_ind, tournsize):
    """Selection function."""
    chosen = []    
    for i in range(n_ind):
        aspirants = [random.choice(pop) for j in range(tournsize)]
        chosen.append(min(aspirants, key=attrgetter("fitness")))
    return chosen

def roulette(pop,C_max):
    """Selection function."""
    chosen=[]
    fitness_sum=np.sum([C_max- pop[i].fitness for i in range(len(pop))])      
    sigma_fitness = 0.0
    s = random.random()*fitness_sum
    for i in range(len(pop)):
        sigma_fitness += C_max-pop[i].fitness
        if (sigma_fitness)>=s:
            chosen.append(pop[i])
            break
    #chosens=chosen[:]
    #print("chosens"+str(len(chosens)))
    return chosen

def cxTwoPointCopy(ind1, ind2):
    """Crossover function."""
    size = len(ind1)
    tmp1 = ind1.copy()
    tmp2 = ind2.copy()
    cxpoint1 = random.randint(1, size)
    cxpoint2 = random.randint(1, size-1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else: # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1
    tmp1[cxpoint1:cxpoint2], tmp2[cxpoint1:cxpoint2] = tmp2[cxpoint1:cxpoint2].copy(), tmp1[cxpoint1:cxpoint2].copy()
    return tmp1, tmp2

def evalchild(offspring,n_ind):
    return np.sum([offspring[i].fitness for i in range(n_ind)])/n_ind , np.sum([(offspring[k].fitness - fit_mean)**2 for k in range(n_ind)])/n_ind

def mutFlipBit(ind, indpb):
    """Mutation function."""
    tmp = ind.copy()
    for i in range(len(ind)):
        tmp[i] = type(ind[i])(not ind[i]) if random.random() < indpb else tmp[i]
    return tmp

def keisan(n_ind,pop):
    return np.sum([pop[i].fitness for i in range(n_ind)])    

def get_parameter_from_ind(ind):
    bit_ind=np.array([2**(8-i) for i in range(9)])#array([256,128, 64, 32, 16, 8, 4, 2,1])
    """1峰目"""
    a=0
    Hv_1     =0.010+0.020*np.sum([ind[i]*2**(8-i) for i in range(9)])#0.1<=H<=10.23np.sum(ind[a:a+9]*bit_ind)#
    a+=9
    lambda_1 =0.01+0.015*np.sum([ind[i+a]*2**(8-i) for i in range(9)])#0.01<=H<=7.675np.sum(ind[a:a+9]*bit_ind)#
    a+=9
    cos_1   =2/255*np.sum([ind[i+a]*2**(7-i) for i in range(8)])-1#np.sum(ind[a:a+8]*bit_ind[1:])#
    a+=8
    sin_1   =2/255*np.sum([ind[i+a]*2**(7-i) for i in range(8)])-1#np.sum(ind[a:a+8]*bit_ind[1:])#
    a+=8
    theta_1  =np.arctan2(sin_1,cos_1)*180/np.pi
    omega_1  =0.35+0.015*np.sum([ind[i+a]*2**(6-i) for i in range(7)])#0.35<=omega<=2.255  #np.sum(ind[a:a+7]*bit_ind[2:])#
    a+=7
    S_max1   =1+83/127*np.sum([ind[i+a]*2**(6-i) for i in range(7)])#np.sum(ind[a:a+7]*bit_ind[2:])#
    a+=7
    
    """2峰目"""
    Hv_2     =0.020*np.sum([ind[i+a]*2**(8-i) for i in range(9)])#0<=H<=10.22#np.sum(ind[a:a+9]*bit_ind)#
    a+=9
    lambda_2 =0.01+0.015*np.sum([ind[i+a]*2**(8-i) for i in range(9)])#0.01<=H<=7.675#np.sum(ind[a:a+9]*bit_ind)#
    a+=9
    cos_2   =2/255*np.sum([ind[i+a]*2**(7-i) for i in range(8)])-1#np.sum(ind[a:a+8]*bit_ind[1:])#
    a+=8
    sin_2   =2/255*np.sum([ind[i+a]*2**(7-i) for i in range(8)])-1#np.sum(ind[a:a+8]*bit_ind[1:])#
    a+=8
    theta_2  =np.arctan2(sin_2,cos_2)*180/np.pi
    omega_2  =0.35+0.015*np.sum([ind[i+a]*2**(6-i) for i in range(7)])#0.35<=omega<=2.255#np.sum(ind[a:a+7]*bit_ind[2:])#
    a+=7
    S_max2      =1+83/127*np.sum([ind[i+a]*2**(6-i) for i in range(7)])#np.sum(ind[a:a+7]*bit_ind[2:])#
    
    return Hv_1,cos_1,sin_1,lambda_1,omega_1,Hv_2,cos_2,sin_2,lambda_2,omega_2,S_max1,S_max2

def def_S_max(H,omega_m):
    HL=H/(1.56*4*math.pi**2/omega_m**2)
    """波集中度s1の周波数依存"""
    if H==0:
        S_max=0
    elif HL<0.026:
        log_10=0.4444*math.log10(HL)+0.5849
        a=10**(log_10)
        S_max=12.99*a**(-2.5)
        
    elif HL>=0.026:
        log10_gF=(0.1507-math.sqrt(0.1507**2+4*0.005882*0.8789+4*0.005882*math.log10(HL)))/(2*0.005882)
        gF=10**log10_gF
        gT=1.37*(1-1/(1+0.008*gF**(1/3))**5)
        S_max=12.99*gT**2.5
    
    if S_max >=84:
        S_max=84
    return S_max

In [5]:
SeaNum=0
for seastate in wavedata:
    random.seed()
    parameters= np.array([esti_para[SeaNum][0], esti_para[SeaNum][1],esti_para[SeaNum][2],esti_para[SeaNum][3],esti_para[SeaNum][4],esti_para[SeaNum][6],
                          esti_para[SeaNum][7],esti_para[SeaNum][8],esti_para[SeaNum][9],esti_para[SeaNum][10],esti_para[SeaNum][11],esti_para[SeaNum][13] ])
    
    """ Step1 : 母集団作成."""
    pop = create_pop(n_ind, n_gene)
    set_fitness(evalOneMax, pop)  #各個体について評価
    fit_sum = keisan(n_ind,pop)  #適応度の合計値を計算
    best_ind = min(pop, key=attrgetter("fitness"))  #最良個体の適応度
    
    #Hv_1,cos1,sin1,lambda_1,omega_1,Hv_2,cos2,sin2,lambda_2,omega_2,S_max1,S_max2=get_parameter_from_ind(best_ind)

    # --- 世代分の繰り返し.
    for g in range(NGEN):     
        """ Step2 : 選択."""
        parents,pop = randomselection(n_ind,pop,n_parents)
        fit_sum=fit_sum-parents[0].fitness-parents[1].fitness
        children = parents.copy()
  
        """ Step3 : 交叉."""
        crossover = []
        for child1, child2 in zip(children[::2], children[1::2]):
            if random.random() < CXPB:
                child1, child2 = cxTwoPointCopy(child1, child2)
                child1.fitness = None
                child2.fitness = None
            crossover.append(child1)
            crossover.append(child2)
        children = crossover[:]


        """ Step4 : 突然変異."""
        mutant = []
        for mut in (children):
            if random.random() < MUTPB:
                mut = mutFlipBit(mut, indpb=MUTINDPB)
                mut.fitness = None
            mutant.append(mut)

        children = mutant[:]
        set_fitness(evalOneMax, children)

        
        """Step5 : 生存選択"""
        family = children+parents
        offspring = []
        
        family_sorted = sorted(family, key=attrgetter("fitness")) #ここで，誤差が小さい順に並べる
        offspring=roulette(family_sorted[1:],family_sorted[3].fitness)
        offspring+=family_sorted[:1]

        
        """ 次世代を現世代に更新."""
        pop = pop + offspring
        fit_sum = fit_sum+offspring[0].fitness+offspring[1].fitness
        fitness_mean =fit_sum/n_ind #,fit_var
        """ Print best fitness in the population."""
        best_ind = min(pop, key=attrgetter("fitness"))
        f_max = best_ind.fitness
        CXPB,MUTINDPB = adjustrating(CXPB,MUTINDPB,CXPB_MAX,CXPB_MIN,M_MAX,M_MIN,f,f_max,fitness_mean)
    
    Hv_1,cos1,sin1,lambda_1,omega_1,Hv_2,cos2,sin2,lambda_2,omega_2,S_max1,S_max2=get_parameter_from_ind(best_ind)
    theta_1=np.arctan2(sin1,cos1)*180/np.pi
    theta_2=np.arctan2(sin2,cos2)*180/np.pi
    if omega_1 < omega_2:
        writing.write(str('%03.8f' %(Hv_1))+","+str('%03.8f' %(theta_1))+","+str('%03.8f' %(lambda_1))+","+str('%03.8f' %(omega_1))+","+str('%03.8f' %(S_max1))+",")
        writing.write(str('%03.8f' %(Hv_2))+","+str('%03.8f' %(theta_2))+","+str('%03.8f' %(lambda_2))+","+str('%03.8f' %(omega_2))+","+str('%03.8f' %(S_max2))+",")
    elif omega_1 == omega_2 :
        if Hv_1 >= Hv_2 :
            writing.write(str('%03.8f' %(Hv_1))+","+str('%03.8f' %(theta_1))+","+str('%03.8f' %(lambda_1))+","+str('%03.8f' %(omega_1))+","+str('%03.8f' %(S_max1))+",")
            writing.write(str('%03.8f' %(Hv_2))+","+str('%03.8f' %(theta_2))+","+str('%03.8f' %(lambda_2))+","+str('%03.8f' %(omega_2))+","+str('%03.8f' %(S_max2))+",")
        else:
            writing.write(str('%03.8f' %(Hv_2))+","+str('%03.8f' %(theta_2))+","+str('%03.8f' %(lambda_2))+","+str('%03.8f' %(omega_2))+","+str('%03.8f' %(S_max2))+",")
            writing.write(str('%03.8f' %(Hv_1))+","+str('%03.8f' %(theta_1))+","+str('%03.8f' %(lambda_1))+","+str('%03.8f' %(omega_1))+","+str('%03.8f' %(S_max1))+",")
    else:
        writing.write(str('%03.8f' %(Hv_2))+","+str('%03.8f' %(theta_2))+","+str('%03.8f' %(lambda_2))+","+str('%03.8f' %(omega_2))+","+str('%03.8f' %(S_max2))+",")
        writing.write(str('%03.8f' %(Hv_1))+","+str('%03.8f' %(theta_1))+","+str('%03.8f' %(lambda_1))+","+str('%03.8f' %(omega_1))+","+str('%03.8f' %(S_max1))+",")    
    writing.write(str('%03.8f' %(best_ind.fitness)))
    writing.write('\n')
    SeaNum+=1
    print(str(len(wavedata))+"海象中 "+str(SeaNum)+"番目の海象で誤差は "+str(best_ind.fitness))
writing.close()

3000海象中 1番目の海象で誤差は 19.62249666075313
3000海象中 2番目の海象で誤差は 27.55511070229621
3000海象中 3番目の海象で誤差は 32.21285717731635
3000海象中 4番目の海象で誤差は 31.41310803215186
3000海象中 5番目の海象で誤差は 24.764146611944316
3000海象中 6番目の海象で誤差は 13.917598808268307
3000海象中 7番目の海象で誤差は 8.542192459205548
3000海象中 8番目の海象で誤差は 7.422552444197821
3000海象中 9番目の海象で誤差は 4.4422931034686215
3000海象中 10番目の海象で誤差は 3.3498604401730656
3000海象中 11番目の海象で誤差は 2.681673394470066
3000海象中 12番目の海象で誤差は 1.4746488883817084
3000海象中 13番目の海象で誤差は 1.3936398678457893
3000海象中 14番目の海象で誤差は 0.6892827059493328
3000海象中 15番目の海象で誤差は 0.6833988156069487
3000海象中 16番目の海象で誤差は 0.8218936911902581
3000海象中 17番目の海象で誤差は 1.409255151956651
3000海象中 18番目の海象で誤差は 2.4501554931537846
3000海象中 19番目の海象で誤差は 3.4585976604396733
3000海象中 20番目の海象で誤差は 3.4080123900524866
3000海象中 21番目の海象で誤差は 9.802786248110063
3000海象中 22番目の海象で誤差は 9.555843321361115
3000海象中 23番目の海象で誤差は 9.346018978197977
3000海象中 24番目の海象で誤差は 5.967204279039931
3000海象中 25番目の海象で誤差は 4.899183947401441
3000海象中 26番目の海象で誤差は 4.296756884518047
3000海象中 2

3000海象中 210番目の海象で誤差は 0.009100969549090622
3000海象中 211番目の海象で誤差は 0.011622561959500432
3000海象中 212番目の海象で誤差は 0.009892809997347445
3000海象中 213番目の海象で誤差は 0.013128125285682189
3000海象中 214番目の海象で誤差は 0.017427661287251903
3000海象中 215番目の海象で誤差は 0.016112259506270096
3000海象中 216番目の海象で誤差は 0.02822986812114819
3000海象中 217番目の海象で誤差は 0.06028552802416476
3000海象中 218番目の海象で誤差は 0.0897954258694113
3000海象中 219番目の海象で誤差は 0.10090265841712331
3000海象中 220番目の海象で誤差は 0.09536570344821288
3000海象中 221番目の海象で誤差は 0.22654079931938514
3000海象中 222番目の海象で誤差は 0.17139934391588782
3000海象中 223番目の海象で誤差は 0.33730735811496076
3000海象中 224番目の海象で誤差は 0.588298006217954
3000海象中 225番目の海象で誤差は 0.3994455059454717
3000海象中 226番目の海象で誤差は 1.422048218954902
3000海象中 227番目の海象で誤差は 6.004027916361539
3000海象中 228番目の海象で誤差は 7.824205240700426
3000海象中 229番目の海象で誤差は 13.182528584524544
3000海象中 230番目の海象で誤差は 15.834747046809875
3000海象中 231番目の海象で誤差は 15.886808595275808
3000海象中 232番目の海象で誤差は 19.375878409958002
3000海象中 233番目の海象で誤差は 21.000356038150855
3000海象中 234番目の海象で誤差は 18.4

3000海象中 415番目の海象で誤差は 0.0848838969898443
3000海象中 416番目の海象で誤差は 0.104152452294345
3000海象中 417番目の海象で誤差は 0.13417183978225367
3000海象中 418番目の海象で誤差は 0.13022113075376168
3000海象中 419番目の海象で誤差は 0.13769433048259058
3000海象中 420番目の海象で誤差は 0.1378627019603994
3000海象中 421番目の海象で誤差は 0.12087835288408641
3000海象中 422番目の海象で誤差は 0.18032928867150622
3000海象中 423番目の海象で誤差は 0.1893686226557879
3000海象中 424番目の海象で誤差は 0.1956599361560722
3000海象中 425番目の海象で誤差は 0.17199191865697483
3000海象中 426番目の海象で誤差は 0.09706935196433532
3000海象中 427番目の海象で誤差は 0.1445971981123311
3000海象中 428番目の海象で誤差は 0.14728023554669462
3000海象中 429番目の海象で誤差は 0.40352670873713864
3000海象中 430番目の海象で誤差は 0.3823644409279281
3000海象中 431番目の海象で誤差は 0.5881703530956566
3000海象中 432番目の海象で誤差は 0.38965148977178643
3000海象中 433番目の海象で誤差は 0.23055505305227916
3000海象中 434番目の海象で誤差は 0.13012612145261573
3000海象中 435番目の海象で誤差は 0.09125435180620148
3000海象中 436番目の海象で誤差は 0.0971775555386886
3000海象中 437番目の海象で誤差は 0.057674142033073514
3000海象中 438番目の海象で誤差は 0.07253250655935649
3000海象中 439番目の海象で誤差は 0.03

3000海象中 618番目の海象で誤差は 66.78150022047254
3000海象中 619番目の海象で誤差は 79.83711824165322
3000海象中 620番目の海象で誤差は 76.4740232415386
3000海象中 621番目の海象で誤差は 63.63477836840893
3000海象中 622番目の海象で誤差は 81.91907515661495
3000海象中 623番目の海象で誤差は 60.75364194426507
3000海象中 624番目の海象で誤差は 67.78079769096269
3000海象中 625番目の海象で誤差は 62.051610353192906
3000海象中 626番目の海象で誤差は 85.37105986225367
3000海象中 627番目の海象で誤差は 62.18624651501641
3000海象中 628番目の海象で誤差は 65.69995726809255
3000海象中 629番目の海象で誤差は 61.316772900468976
3000海象中 630番目の海象で誤差は 80.68162111405202
3000海象中 631番目の海象で誤差は 82.72400958941168
3000海象中 632番目の海象で誤差は 64.20301764662267
3000海象中 633番目の海象で誤差は 70.34806816139532
3000海象中 634番目の海象で誤差は 77.03690697853845
3000海象中 635番目の海象で誤差は 70.87036023170809
3000海象中 636番目の海象で誤差は 80.15254324137815
3000海象中 637番目の海象で誤差は 62.11875146639631
3000海象中 638番目の海象で誤差は 76.9464794702323
3000海象中 639番目の海象で誤差は 65.78118368317092
3000海象中 640番目の海象で誤差は 75.02961138024556
3000海象中 641番目の海象で誤差は 74.43876724637651
3000海象中 642番目の海象で誤差は 78.49208966295149
3000海象中 643番目の海象で誤差は 64.1

3000海象中 822番目の海象で誤差は 0.5606687226561653
3000海象中 823番目の海象で誤差は 0.5003024655930595
3000海象中 824番目の海象で誤差は 0.42933394173491074
3000海象中 825番目の海象で誤差は 0.34478441701870116
3000海象中 826番目の海象で誤差は 0.33042185985675815
3000海象中 827番目の海象で誤差は 0.2959612129741291
3000海象中 828番目の海象で誤差は 0.2846233997726392
3000海象中 829番目の海象で誤差は 0.3028491258616509
3000海象中 830番目の海象で誤差は 0.24133494057784644
3000海象中 831番目の海象で誤差は 0.3528503660222722
3000海象中 832番目の海象で誤差は 0.2354070878102479
3000海象中 833番目の海象で誤差は 0.2429046965048236
3000海象中 834番目の海象で誤差は 0.27471277792875937
3000海象中 835番目の海象で誤差は 0.23139404161555205
3000海象中 836番目の海象で誤差は 0.28569811343570245
3000海象中 837番目の海象で誤差は 0.31442987175741066
3000海象中 838番目の海象で誤差は 0.3259351094648741
3000海象中 839番目の海象で誤差は 0.36132152112461596
3000海象中 840番目の海象で誤差は 0.35762100872533714
3000海象中 841番目の海象で誤差は 0.502457148897923
3000海象中 842番目の海象で誤差は 0.7319459785138883
3000海象中 843番目の海象で誤差は 0.6728119746430553
3000海象中 844番目の海象で誤差は 0.7817059456717095
3000海象中 845番目の海象で誤差は 0.922693113627841
3000海象中 846番目の海象で誤差は 1.811037257

...

3000海象中 2213番目の海象で誤差は 0.0001265956246162809
3000海象中 2214番目の海象で誤差は 8.650047696102452e-05
3000海象中 2215番目の海象で誤差は 9.496890035079498e-05
3000海象中 2216番目の海象で誤差は 0.000100150097032388
3000海象中 2217番目の海象で誤差は 7.560595475510246e-05
3000海象中 2218番目の海象で誤差は 0.0001274591386212807
3000海象中 2219番目の海象で誤差は 0.0001034690294526539
3000海象中 2220番目の海象で誤差は 3.7621159704233864e-05
3000海象中 2221番目の海象で誤差は 8.91093351211785e-05
3000海象中 2222番目の海象で誤差は 4.896405390429409e-05
3000海象中 2223番目の海象で誤差は 6.380322823612718e-05
3000海象中 2224番目の海象で誤差は 6.171298372466219e-05
3000海象中 2225番目の海象で誤差は 6.907288500013643e-05
3000海象中 2226番目の海象で誤差は 8.113073515566123e-05
3000海象中 2227番目の海象で誤差は 6.237272484813169e-05
3000海象中 2228番目の海象で誤差は 0.00018112333699911535
3000海象中 2229番目の海象で誤差は 0.00017916620794985113
3000海象中 2230番目の海象で誤差は 0.0007435877361317293
3000海象中 2231番目の海象で誤差は 0.0015593028200238588
3000海象中 2232番目の海象で誤差は 0.0021176782856194855
3000海象中 2233番目の海象で誤差は 0.0059439833754036655
3000海象中 2234番目の海象で誤差は 0.008123302925073473
3000海象中 2235番目の海象で誤差は 0.01303992

3000海象中 2404番目の海象で誤差は 0.07486776420399764
3000海象中 2405番目の海象で誤差は 0.05561922927081466
3000海象中 2406番目の海象で誤差は 0.0760644218966445
3000海象中 2407番目の海象で誤差は 0.036539198822944374
3000海象中 2408番目の海象で誤差は 0.02866273655952815
3000海象中 2409番目の海象で誤差は 0.018094892466829243
3000海象中 2410番目の海象で誤差は 0.015691285960191072
3000海象中 2411番目の海象で誤差は 0.01340704644197942
3000海象中 2412番目の海象で誤差は 0.010239906853478949
3000海象中 2413番目の海象で誤差は 0.009919828640292493
3000海象中 2414番目の海象で誤差は 0.005908601357920254
3000海象中 2415番目の海象で誤差は 0.004644115788411189
3000海象中 2416番目の海象で誤差は 0.005196455177925922
3000海象中 2417番目の海象で誤差は 0.0023133066054539277
3000海象中 2418番目の海象で誤差は 0.0014689674321955249
3000海象中 2419番目の海象で誤差は 0.0016718178200593806
3000海象中 2420番目の海象で誤差は 0.0013631243311629001
3000海象中 2421番目の海象で誤差は 0.001264264290456032
3000海象中 2422番目の海象で誤差は 0.0007689798582058993
3000海象中 2423番目の海象で誤差は 0.0006828430026983159
3000海象中 2424番目の海象で誤差は 0.0006990118870210428
3000海象中 2425番目の海象で誤差は 0.000481766382572087
3000海象中 2426番目の海象で誤差は 0.0002651340172584296
3000海象中 2

3000海象中 2593番目の海象で誤差は 0.0003321042112928776
3000海象中 2594番目の海象で誤差は 0.0003357827292886716
3000海象中 2595番目の海象で誤差は 0.0003808018846431905
3000海象中 2596番目の海象で誤差は 0.0004884899657392719
3000海象中 2597番目の海象で誤差は 0.0005740422880566237
3000海象中 2598番目の海象で誤差は 0.0005156445225260745
3000海象中 2599番目の海象で誤差は 0.0005077986200547396
3000海象中 2600番目の海象で誤差は 0.0027743025978140172
3000海象中 2601番目の海象で誤差は 0.0035561158723708642
3000海象中 2602番目の海象で誤差は 0.00222565653808309
3000海象中 2603番目の海象で誤差は 0.003168962696594647
3000海象中 2604番目の海象で誤差は 0.0014866588512680284
3000海象中 2605番目の海象で誤差は 0.0018873817048936637
3000海象中 2606番目の海象で誤差は 0.0015785256971689772
3000海象中 2607番目の海象で誤差は 0.0014950004155487545
3000海象中 2608番目の海象で誤差は 0.0011807407326854674
3000海象中 2609番目の海象で誤差は 0.0009477490578098139
3000海象中 2610番目の海象で誤差は 0.0008448464823358249
3000海象中 2611番目の海象で誤差は 0.000668089033196003
3000海象中 2612番目の海象で誤差は 0.0007632758332063335
3000海象中 2613番目の海象で誤差は 0.00048651824410729863
3000海象中 2614番目の海象で誤差は 0.0008343773371178821
3000海象中 2615番目の海象で誤差は 0.00041301969

3000海象中 2792番目の海象で誤差は 31.926418242982823
3000海象中 2793番目の海象で誤差は 36.970791583794664
3000海象中 2794番目の海象で誤差は 31.862711949173534
3000海象中 2795番目の海象で誤差は 32.0560021421246
3000海象中 2796番目の海象で誤差は 30.21893535767404
3000海象中 2797番目の海象で誤差は 29.57833699322618
3000海象中 2798番目の海象で誤差は 29.57964255248161
3000海象中 2799番目の海象で誤差は 29.02291596201493
3000海象中 2800番目の海象で誤差は 29.0358099842688
3000海象中 2801番目の海象で誤差は 32.64729245167953
3000海象中 2802番目の海象で誤差は 31.891788994517675
3000海象中 2803番目の海象で誤差は 34.28814628060073
3000海象中 2804番目の海象で誤差は 32.61290183856493
3000海象中 2805番目の海象で誤差は 34.24688913572343
3000海象中 2806番目の海象で誤差は 36.31415928905644
3000海象中 2807番目の海象で誤差は 36.86521464990985
3000海象中 2808番目の海象で誤差は 37.196101303004816
3000海象中 2809番目の海象で誤差は 36.755373852162904
3000海象中 2810番目の海象で誤差は 38.05324788466034
3000海象中 2811番目の海象で誤差は 40.91089013870718
3000海象中 2812番目の海象で誤差は 37.015040596497315
3000海象中 2813番目の海象で誤差は 25.19935884286535
3000海象中 2814番目の海象で誤差は 22.483747912541883
3000海象中 2815番目の海象で誤差は 47.644130378816385
3000海象中 2816番目の海象で誤差は 50.16158349

3000海象中 2991番目の海象で誤差は 13.215445529332136
3000海象中 2992番目の海象で誤差は 14.045886935859071
3000海象中 2993番目の海象で誤差は 12.521637386242135
3000海象中 2994番目の海象で誤差は 15.181938529918366
3000海象中 2995番目の海象で誤差は 27.343531870101952
3000海象中 2996番目の海象で誤差は 34.48797020316135
3000海象中 2997番目の海象で誤差は 39.371565183482076
3000海象中 2998番目の海象で誤差は 36.112954690744886
3000海象中 2999番目の海象で誤差は 57.69009747991916
3000海象中 3000番目の海象で誤差は 42.29261052321151


In [7]:
writing.close()